In [1]:
from pathlib import Path
import polars as pl
from qwen_model import QwenChatInference

In [2]:
file = Path('/home/kevin/Downloads/Temp Query 3_20241002-164744.csv')
df = pl.read_csv(file)
print(df.shape)
df.head()

(182940, 2)


shop_id,comment
i64,str
23829086,"""Sản phẩm đủ màu. Nhưng bị lem …"
23829086,"""Hang nhat ma sao ma vach lai l…"
23829086,""" Chất lượng sản phẩm tốt"""
23829086,"""Hàng đẹp ạ, nhưng size hơi to …"
23829086,"""Áo lót bông chứ không phải lôn…"


In [16]:
df

shop_id,comment
i64,str
23829086,"""Sản phẩm đủ màu. Nhưng bị lem …"
23829086,"""Hang nhat ma sao ma vach lai l…"
23829086,""" Chất lượng sản phẩm tốt"""
23829086,"""Hàng đẹp ạ, nhưng size hơi to …"
23829086,"""Áo lót bông chứ không phải lôn…"
…,…
23829086,"""Chuẩn bị xài thử. Hihi"""
23829086,"""Chất lượng sản phẩm tuyệt vời,…"
23829086,"""Bộ cắt móng gọn nhẹ, tiện sử d…"


In [3]:
', '.join(df['comment'][:100])

'Sản phẩm đủ màu. Nhưng bị lem bẩn. Sau shop chú ý kiểm tra trước khi đóng hàng, Hang nhat ma sao ma vach lai la trung quoc nhi,  Chất lượng sản phẩm tốt, Hàng đẹp ạ, nhưng size hơi to với em. Rất ưng ý. Cảm ơn shop. Sẽ ủng hộ nếu có nhu câu thêm, Áo lót bông chứ không phải lông, hơi mỏng xíu, đường may chắc chắn, shop phục vụ tốt, Quai đeo hơi dão,  Chất lượng sản phẩm tuyệt vời giao hàng rất chậm, Phù hợp giá tiền, Dép không giống hình. Logo facebook??, Giao size 1 thành size 2 rồi shop ơi, , chất liệu bt phù hợp giá tiền, Chất vải tạm được,  Thời gian giao hàng nhanh, Miếng lót k cố định\xa0, Thời gian giao hàng chậm, Bộ cắt móng tay hết hàng mà shop cũng không nt cho mình biết để hôm 9.9 mình mua chổ khác.,  Chất lượng sản phẩm tạm được, Giao hàng chậm, đặt đúng 1 tuần mới nhận dc, Vải mỏng\xa0, Khăn bị bẩn 1 chút. Đóng gói sản phẩm chắc chắn, Mình mua lần thứ 2 nhưng chất Hình như không giống nhau , E đặt số 1 mà thấy trên nhãn ghi toàn số 2, Chất lượng sản phẩm và giá cả tốt. Tuy

In [4]:
prompt = f"""
Objective: Create a concise yet comprehensive summary of customer product reviews from an e-commerce platform, 
highlighting key insights and trends to inform potential buyers and provide valuable feedback to the seller.

1. Quantitative Analysis
- Identify the most common star ratings
- Calculate the percentage of positive (4-5 stars) vs. negative (1-2 stars) reviews
- Note any trends in ratings over time (improving, declining, or stable)

2. Key Themes
- Identify 3-5 most frequently mentioned positive aspects of the product
- Identify 3-5 most frequently mentioned negative aspects or concerns
- Highlight any unique or standout comments that provide valuable insights

3. Product Features
- List the most discussed product features
- For each key feature, provide a brief summary of customer opinions (positive and negative)

4. Customer Experience
- Summarize comments about the overall customer experience (e.g., ease of use, satisfaction level)
- Note any frequently mentioned comparisons to similar products or brands

5. Quality and Durability
- Summarize feedback related to product quality
- Highlight any common issues or praises regarding product longevity and durability

6. Value for Money
- Summarize opinions on the product's pricing
- Identify whether customers generally feel the product is worth its cost

7. Customer Service and Seller Performance
- If applicable, summarize feedback about the seller's customer service
- Note any common themes regarding shipping, packaging, or seller communication

8. Pros and Cons
- Create a concise list of pros and cons based on the review analysis

9. Potential Improvements
- Identify areas where customers frequently suggest improvements

10. Reviewer Demographics
- If discernible, note any patterns in the types of customers leaving reviews (e.g., age groups, usage patterns)

11. Verify Purchase Status
- Note the percentage of reviews from verified purchases, if this information is available

12. Review Quality
- Comment on the overall quality and helpfulness of the reviews
- Highlight any particularly detailed or insightful reviews

13. Recent Trends
- Summarize any noticeable trends in the most recent reviews (last 1-3 months)

14. Visual Content
- Mention the presence and usefulness of customer-submitted photos or videos

15. Keywords and Phrases
- List frequently used words or phrases that capture the essence of customer sentiment

Final Summary
Provide a 2-3 sentence overall summary that captures the general customer consensus about the product, 
its standout features, and any major caveats potential buyers should be aware of.

Note: 
When summarizing, strive for objectivity and balance. Represent both positive and negative 
feedback proportionally to their presence in the reviews. Avoid overgeneralizing and use qualifying language (e.g., "many customers," "some users") when appropriate.

Remember, your goal is to create a structured summary that quickly gives potential customers the most important information about the product, helping them make informed purchasing decisions, while providing the data in a format that's easy to process programmatically.
Summarize the product description below, delimited by triple 
backticks.

Reviews: ```{', '.join(df['comment'][:100])}```
"""

In [5]:
print(prompt)


Objective: Create a concise yet comprehensive summary of customer product reviews from an e-commerce platform, 
highlighting key insights and trends to inform potential buyers and provide valuable feedback to the seller.

1. Quantitative Analysis
- Identify the most common star ratings
- Calculate the percentage of positive (4-5 stars) vs. negative (1-2 stars) reviews
- Note any trends in ratings over time (improving, declining, or stable)

2. Key Themes
- Identify 3-5 most frequently mentioned positive aspects of the product
- Identify 3-5 most frequently mentioned negative aspects or concerns
- Highlight any unique or standout comments that provide valuable insights

3. Product Features
- List the most discussed product features
- For each key feature, provide a brief summary of customer opinions (positive and negative)

4. Customer Experience
- Summarize comments about the overall customer experience (e.g., ease of use, satisfaction level)
- Note any frequently mentioned comparison

In [6]:
q = QwenChatInference(flash_attention_2=False, speed_up=True)

[QwenChat Infer] Device: cuda:0, Flash Attention 2: False, Speedup: True


In [12]:
output = q.generate(prompt, verbose=True, max_new_tokens=1024)

[Qwen Chat] 
Time: 5s 
Prompt: 2800 tokens, 600.44 tokens-per-secs


In [13]:
print(output)

### Summary of Customer Product Reviews

#### Overview:
The product has been reviewed by multiple customers, covering various aspects such as color retention, fit, texture, price, and more. The majority of reviews highlight positive aspects like good quality, durability, and value for money, while also noting some negatives like minor issues with sizing and color consistency.

#### Positive Aspects:
1. **Color Retention**: Many customers appreciate the product’s ability to retain colors well after being worn out, indicating durability and comfort.
   
2. **Fit and Texture**: Customers praise the product’s comfort and fit, especially for babies who need support during playtime.

3. **Price**: The product is considered very affordable compared to other baby products on the market.

4. **Value for Money**: Customers find the product to be worth its cost, even though it might not be the best buy for parents.

#### Negative Aspects:
1. **Size Issues**: Some customers report issues with sizi

In [17]:
prompt = f"""
Objective: Create a concise yet comprehensive summary of customer product reviews from an e-commerce platform, 
highlighting key insights and trends to inform potential buyers and provide valuable feedback to the seller.

1. Customer Experience
- Summarize comments about the overall customer experience:
    - Product quality
    - Shipping
    - Price
    - Size
- Note any frequently mentioned comparisons to similar products or brands

2. Keywords and Phrases
- List frequently used words or phrases that capture the essence of customer sentiment

Note: 
When summarizing, strive for objectivity and balance. Represent both positive and negative 
feedback proportionally to their presence in the reviews. Avoid overgeneralizing and use qualifying language (e.g., "many customers," "some users") when appropriate.

Remember, your goal is to create a structured summary that quickly gives potential customers the most important information about the product, helping them make informed purchasing decisions, while providing the data in a format that's easy to process programmatically.
Summarize the product description below, delimited by triple 
backticks.

Reviews: ```{', '.join(df['comment'][:100])}```
"""

output = q.generate(prompt, verbose=True, max_new_tokens=1024)
print(output)

[Qwen Chat] 
Time: 4s 
Prompt: 2374 tokens, 604.59 tokens-per-secs
### Objective Summary

#### 1. **Customer Experience**
- **Overall Quality:** The product has good color, packaging, and sizing.
- **Shipping:** The shipping was delayed due to the size being too small.
- **Price:** The price seems reasonable but not excessively high compared to other retailers.
- **Size:** The size fit well with the child’s age.

#### 2. **Keywords and Phrases**
- **Comparison:** "Product quality", "Shipping delay", "Size", "Price"
- **Feedback:** "Lemme bẩn", "Shops are often wrong", "Too big", "Not as good as expected"

---

### Detailed Summary

#### 1. **Customer Experience**

**Overall Quality:** The product meets basic expectations. It looks good and fits well with the child's size.

**Shipping:** The shipping was delayed because the size was too small. This issue was resolved promptly after the initial shipment.

**Price:** The price seems reasonable but not excessively high compared to other re

In [27]:
prompt = f"""
Objective: Develop a systematic approach to identify potentially fake or manipulated reviews on e-commerce platforms and review sites, ensuring a more accurate representation of genuine customer experiences.

1. Review Content Analysis

Look for generic language: Vague praise or criticism without specific details
Check for repetitive phrases: Similar wording across multiple reviews may indicate copy-pasting
Assess detail level: Lack of product-specific information or personal experiences
Identify irrelevant information: Reviews focusing on unrelated aspects or containing off-topic content
Evaluate writing style: Overly formal or marketing-like language can be suspicious

2. Temporal Patterns

Identify review bursts: Sudden influxes of positive or negative reviews in a short time frame
Compare to product launch dates: Many reviews immediately after product release can be suspicious
Check for time zone inconsistencies: Reviews posted at odd hours for the supposed location of the reviewer

3. Linguistic Analysis

Identify non-native language patterns: Awkward phrasing or grammar inconsistent with the reviewer's supposed origin
Look for excessive use of superlatives: Overuse of words like "best," "amazing," "worst," etc.
Check for brand-specific language: Use of trademark names or marketing slogans that typical customers might not use

4. Sentiment vs. Rating Mismatch
Compare written sentiment to star rating: Identify discrepancies between the review text and the given rating

5. Competitor Mentions
Identify reviews that excessively praise or criticize competitor products

Final Assessment
Combine these factors to create a "suspicion score" for each review. Reviews with multiple red flags should be flagged for further investigation or potential removal.
Remember: No single factor definitively proves a review is fake. Always consider the full context and be cautious about falsely accusing genuine reviews. The goal is to identify patterns and suspicious elements that warrant closer examination.

Identify the product description below, delimited by triple 
backticks.

Reviews: ```{', '.join(df['comment'][:100])}```
"""

output = q.generate(prompt, verbose=True, max_new_tokens=512)
print(output)

[Qwen Chat] 
Time: 4s 
Prompt: 2540 tokens, 631.64 tokens-per-secs
Based on the analysis of the provided product description, I have identified several potential issues:

1. **Generic Language**: The review contains vague praise and criticisms without specific details, which suggests it could be fake or manipulated.

2. **Repetitive Phrases**: The review uses similar wording across multiple reviews, indicating copying behavior.

3. **Lack of Product Information**: The review focuses on unrelated aspects such as the size of the product and its appearance.

4. **Unrelated Content**: The review mentions other products (e.g., clothing) but does not mention any real product comparisons.

5. **Writing Style**: The review has an overly formal tone and includes marketing-style language, which can be suspicious.

6. **Brand-Specific Language**: The review references specific brands ("Baby Leo") but does not provide enough context to determine if this is genuine.

7. **Non-Native Language Patter

In [24]:
df['comment'][0]

'Sản phẩm đủ màu. Nhưng bị lem bẩn. Sau shop chú ý kiểm tra trước khi đóng hàng'